<a href="https://colab.research.google.com/github/dylanesq/2-Months-Project-LLM/blob/main/LlamaIndexV2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index

#Loading

 **python-docx2txt**
A pure python-based utility to extract text from docx files.

The code is taken and adapted from python-docx. It can however also extract text from header, footer and hyperlinks. It can now also extract images.

In [ ]:
pip install docx2txt

Try using raw github dump for loading

In [ ]:
#from llama_index.readers.file.docs import DocxReader
from llama_index.core import SimpleDirectoryReader
#docx_reader = DocxReader()
#document_1 = docx_reader.load_data('/content/drive/MyDrive/002MONTHSPROJECT/')
reader=SimpleDirectoryReader(input_dir="/content/drive/MyDrive/002MONTHSPROJECT/", required_exts=[".docx"])
document_1=reader.load_data()

# Chunking Randomly
[TokenTextSplitter](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.node_parser.TokenTextSplitter.html#tokentextsplitter)

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    separator=" ",
)

*For Review Purpose*

In [ ]:
token_nodes = splitter.get_nodes_from_documents(
    document_1, show_progress=True
)

# LLM Setup
*   [HuggingFaceLLM](https://docs.llamaindex.ai/en/v0.9.48/api_reference/llms/huggingface.html#huggingfacellm)
*   [Llama 3.2 3B
](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)



update : logging without CLI *(contains exposed HF Token , how to fix)*

In [92]:
# @title LOGIN INFO [ HIDDEN]
!huggingface-cli login --token HF_TOKEN --add-to-git-credential

Token is valid (permission: fineGrained).
The token `llama3forAnkan` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama3forAnkan`


In [ ]:
!huggingface-cli whoami
#!huggingface-cli logout

#LLM Init.

In [ ]:
!pip install llama_index.llms.huggingface
from llama_index.llms.huggingface import HuggingFaceLLM

system prompt

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""


Quantization [ 8 bit Working]

[HF Bits&Bytesconfig](https://huggingface.co/docs/transformers/en/main_classes/quantization#transformers.BitsAndBytesConfig)
[Bitnbytes docu](https://pypi.org/project/bitsandbytes/https://)

In [ ]:
!pip install -U bitsandbytes
!pip install accelerate
#!pip install --upgrade transformers accelerate bitsandbytes
from transformers import BitsAndBytesConfig

import torch
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
llm = HuggingFaceLLM(
        context_window=4096,
        max_new_tokens=256,
        generate_kwargs={"temperature": 0.0, "do_sample": False},
        system_prompt=system_prompt,
        tokenizer_name="meta-llama/Llama-3.2-3B-Instruct",
        model_name="meta-llama/Llama-3.2-3B-Instruct",
        device_map="auto", # Ensures the model is loaded on the GPU if available
        # Pass the quantization_config object here
        model_kwargs={
            "torch_dtype": torch.float16, # Recommended for performance on GPU
            "quantization_config": quantization_config # Pass the BitsAndBytesConfig object
        })

In [ ]:
#print("After Quantisation :"(llm.get_memory_footprint())/1e9) need to rework this

# **Embedding & Vectorization**


Vector Imports

In [ ]:
from llama_index.core import VectorStoreIndex
#NOT REQUIRED SimpleDirectoryReader, ServiceContext  #Vector store index is for indexing the vector


Embedding Imports
* [HuggingFaceEmbeddings](https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface/)  
*   [LAngchain HF Embedding](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html#huggingfaceembeddings)
*   [SBERT](https://www.sbert.net/)
*   [Llamindex embedding](https://docs.llamaindex.ai/en/stable/api_reference/embeddings/huggingface/)





In [ ]:
!pip install llama-index-embeddings-huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Set up the HuggingFaceEmbedding class with the required model to use with llamaindex core.
embed_model  = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

# Service Context/Settings

Set Up
Edit  : ServiceContext is deprecated.

*   Use llama_index.settings.Settings instead
*   [Migration to Settings Llama Index](https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/)


In [ ]:
# from llama_index.core import ServiceContext this is obsolote
from llama_index.core import Settings
Settings.llm=llm
Settings.embed_model=embed_model
Settings.node_parser=splitter

#Indexing

In [ ]:

index=VectorStoreIndex.from_documents(document_1)#no longer required to have servicecontext
#query_engine=index.as_query_engine(topK=5)

#Query Engine



*   [Querying LlamaIndex](https://docs.llamaindex.ai/en/stable/module_guides/querying/)
*   [Query Engine Module
](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/modules/)


Custom Query engine **[working]**

In [78]:

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
#from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=5)
#postprocessor=SimilarityPostprocessor(similarity_cutoff=0.60)

query_engine=RetrieverQueryEngine(retriever=retriever)
#query_engine=RetrieverQueryEngine(retriever=retriever,node_postprocessors=[postprocessor])

**Vanilla Query Response**

In [77]:
prompt="what is this document about?"

Responses

In [79]:
norank_retrieved_nodes = retriever.retrieve(prompt)
print(f"Retrieved {len(norank_retrieved_nodes)} nodes:")
for i, node in enumerate(norank_retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

Retrieved 2 nodes:

--- Node 1 ---
ante odio, ut interdum tellus imperdiet vitae. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Suspendisse commodo, tellus ac varius egestas, magna nisi condimentum nunc, vitae tincidunt nisl lorem eget eros. Duis tristique ultrices magna, a varius ligula ornare pharetra. Fusce luctus nisl est, ac semper nunc dignissim tempor.

Praesent sed lorem lorem. Donec egestas eros nec rhoncus efficitur. Duis nec mi arcu. Suspendisse in elit elit. Suspendisse potenti. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Quisque mattis metus eu nisi commodo, in semper orci malesuada. Nullam ullamcorper dapibus dolor in egestas.

Aenean faucibus dui sit amet interdum laoreet. Sed ac pretium arcu, at dictum nisl. Donec ullamcorper ac diam et finibus. Suspendisse hendrerit lorem sem, ut mattis ipsum malesuada vel. Morbi ullamcorper elementum viverra. Fusce eget placerat urna, et ornare tortor. Suspendisse facilisis 

In [84]:
response=query_engine.query(prompt)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [85]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
#print(response)

Final Response: This document appears to be a sample text or a
template for a document, likely a business or marketing document,
given the use of Latin phrases such as "Lorem ipsum dolor sit amet"
and the structure of the text. The content is a collection of
paragraphs with varying lengths, each containing a mix of Latin
phrases, words, and phrases in English. The text does not appear to be
a coherent narrative or a formal report, but rather a collection of
phrases and sentences used to fill space or create a visually
appealing layout. Without more context, it is difficult to determine
the specific purpose or content of the document.
______________________________________________________________________
Source Node 1/2
Node ID: 64b3c297-992b-4418-a086-87f38898222e
Similarity: 0.18062013069197172
Text: ante odio, ut interdum tellus imperdiet vitae. Lorem ipsum dolor
sit amet, consectetur adipiscing elit. Suspendisse commodo, tellus ac
varius egestas, magna nisi condimentum nunc, vitae t

# Implement Rerank
[Query Bundle](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.schema.QueryBundle.html#querybundle)

In [89]:
#from llama_index.core.retrievers import VectorIndexRetriever (Already Imported)
from llama_index.core import QueryBundle
from llama_index.core.indices.postprocessor import LLMRerank

#from typing import List

retriever2 = VectorIndexRetriever(
        index=index,
        similarity_top_k=5,
        #vector_store_query_mode="default",

    )

query_bundle = QueryBundle(prompt)

retrieved_nodes = retriever2.retrieve(query_bundle)

reranker = LLMRerank(
            choice_batch_size=5,
            top_n=5,
        )
retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )
query_engine2=RetrieverQueryEngine(retriever=retriever2)
#pprint_response(retrieved_nodes,show_source=True)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [83]:
print(f"Retrieved {len(retrieved_nodes)} nodes:")
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

Retrieved 2 nodes:

--- Node 1 ---
ante odio, ut interdum tellus imperdiet vitae. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Suspendisse commodo, tellus ac varius egestas, magna nisi condimentum nunc, vitae tincidunt nisl lorem eget eros. Duis tristique ultrices magna, a varius ligula ornare pharetra. Fusce luctus nisl est, ac semper nunc dignissim tempor.

Praesent sed lorem lorem. Donec egestas eros nec rhoncus efficitur. Duis nec mi arcu. Suspendisse in elit elit. Suspendisse potenti. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Quisque mattis metus eu nisi commodo, in semper orci malesuada. Nullam ullamcorper dapibus dolor in egestas.

Aenean faucibus dui sit amet interdum laoreet. Sed ac pretium arcu, at dictum nisl. Donec ullamcorper ac diam et finibus. Suspendisse hendrerit lorem sem, ut mattis ipsum malesuada vel. Morbi ullamcorper elementum viverra. Fusce eget placerat urna, et ornare tortor. Suspendisse facilisis 

In [90]:
response2=query_engine2.query(query_bundle)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [91]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response2,show_source=True)
#print(response)

Final Response: This document appears to be a sample text or a
placeholder content, likely used for demonstration or testing
purposes. It does not contain any specific information or content that
can be summarized or analyzed. The text is composed of a series of
paragraphs with varying lengths, and it seems to be a collection of
random sentences and phrases. Without more context or information, it
is difficult to determine the purpose or meaning of this document. The
document's content is not related to any specific topic or subject,
and it does not provide any clear or concise information. The text is
likely a generic placeholder or a sample text used for testing or
demonstration purposes.   Note: The refined answer is based on the
provided context, which is a large block of text that appears to be a
sample document. The context does not provide any specific information
about the document's purpose or content, so the refined answer is
similar to the original answer. If more context we